__[numpy grid](https://github.com/apowers313/roc/blob/master/experiments/2024.08.14-06.56.06-numpy-grid/2024.08.14-06.56.06-numpy-grid.ipynb)__

In [1]:
!date

Wed Aug 14 06:56:23 PDT 2024


In [15]:
import numpy as np

a = np.arange(20).reshape((4, 5))
print(a)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]


In [6]:
for idx, v in np.ndenumerate(a.T):
    print(idx, v)

(0, 0) 0
(0, 1) 5
(0, 2) 10
(0, 3) 15
(1, 0) 1
(1, 1) 6
(1, 2) 11
(1, 3) 16
(2, 0) 2
(2, 1) 7
(2, 2) 12
(2, 3) 17
(3, 0) 3
(3, 1) 8
(3, 2) 13
(3, 3) 18
(4, 0) 4
(4, 1) 9
(4, 2) 14
(4, 3) 19


In [7]:
str(a)

'[[ 0  1  2  3  4]\n [ 5  6  7  8  9]\n [10 11 12 13 14]\n [15 16 17 18 19]]'

# Metadata

In [8]:
# data type metadata
# https://numpy.org/doc/stable/reference/generated/numpy.dtype.metadata.html

dt = np.dtype(float, metadata={"key": "value"})
print("dt.metadata[key]", dt.metadata["key"])

arr = np.array([1, 2, 3], dtype=dt)
arr.dtype.metadata
print("arr.dtype.metadata", arr.dtype.metadata)

dt.metadata[key] value
arr.dtype.metadata {'key': 'value'}


# Subclassing
https://numpy.org/doc/stable/user/basics.subclassing.html

Interoperability instead of Subclassing:
https://numpy.org/doc/stable/user/basics.interoperability.html

Pandas: https://stackoverflow.com/questions/14688306/adding-meta-information-metadata-to-pandas-dataframe

In [16]:
class TestArray(np.ndarray):
    @property
    def groot(self):
        print("I am groot")

In [22]:
b = a.view(TestArray)
print("b", b)
b.groot

try:
    a.groot
except Exception:
    print("a has no groot")

b [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
I am groot
a has no groot


In [24]:
c = b.copy()
c.groot

I am groot


In [28]:
d = b[1:].T
print(d)
d.groot

[[ 5 10 15]
 [ 6 11 16]
 [ 7 12 17]
 [ 8 13 18]
 [ 9 14 19]]
I am groot


In [30]:
isinstance(d, TestArray)

True

In [35]:
import numpy as np


class InfoArray(np.ndarray):

    def __new__(
        subtype, shape, dtype=float, buffer=None, offset=0, strides=None, order=None, info=None
    ):
        print("InfoArray.__new__")
        # Create the ndarray instance of our type, given the usual
        # ndarray input arguments.  This will call the standard
        # ndarray constructor, but return an object of our type.
        # It also triggers a call to InfoArray.__array_finalize__
        obj = super().__new__(subtype, shape, dtype, buffer, offset, strides, order)
        # set the new 'info' attribute to the value passed
        obj.info = info
        # Finally, we must return the newly created object:
        return obj

    def __array_finalize__(self, obj):
        print("InfoArray.__array_finalize__")
        # ``self`` is a new object resulting from
        # ndarray.__new__(InfoArray, ...), therefore it only has
        # attributes that the ndarray.__new__ constructor gave it -
        # i.e. those of a standard ndarray.
        #
        # We could have got to the ndarray.__new__ call in 3 ways:
        # From an explicit constructor - e.g. InfoArray():
        #    obj is None
        #    (we're in the middle of the InfoArray.__new__
        #    constructor, and self.info will be set when we return to
        #    InfoArray.__new__)
        if obj is None:
            return
        # From view casting - e.g arr.view(InfoArray):
        #    obj is arr
        #    (type(obj) can be InfoArray)
        # From new-from-template - e.g infoarr[:3]
        #    type(obj) is InfoArray
        #
        # Note that it is here, rather than in the __new__ method,
        # that we set the default value for 'info', because this
        # method sees all creation of default objects - with the
        # InfoArray.__new__ constructor, but also with
        # arr.view(InfoArray).
        self.info = getattr(obj, "info", None)
        # We do not need to return anything

In [40]:
# explicit constructor
print("--- explicit constructor ---")
obj = InfoArray(shape=(3,))
print("obj:", type(obj), obj)

obj = InfoArray(shape=(3,), info="information")
print("obj:", type(obj), obj)
print("obj info", obj.info)

# new-from-template
print("\n--- new from template ---")
v = obj[1:]
print("v:", type(v), v)
print("v info", v.info)

# view casting
print("\n--- view casting ---")
arr = np.arange(10)
cast_arr = arr.view(InfoArray)
print("cast_arr:", type(cast_arr), cast_arr)
print("cast_arr info", cast_arr.info)

--- explicit constructor ---
InfoArray.__new__
InfoArray.__array_finalize__
obj: <class '__main__.InfoArray'> [1. 2. 3.]
InfoArray.__new__
InfoArray.__array_finalize__
obj: <class '__main__.InfoArray'> [1. 2. 3.]
obj info information

--- new from template ---
InfoArray.__array_finalize__
v: <class '__main__.InfoArray'> [2. 3.]
v info information

--- view casting ---
InfoArray.__array_finalize__
cast_arr: <class '__main__.InfoArray'> [0 1 2 3 4 5 6 7 8 9]
cast_arr info None


In [42]:
import numpy as np


class RealisticInfoArray(np.ndarray):

    def __new__(cls, input_array, info=None):
        print("RealisticInfoArray.__new__")
        # Input array is an already formed ndarray instance
        # We first cast to be our class type
        obj = np.asarray(input_array).view(cls)
        # add the new attribute to the created instance
        obj.info = info
        # Finally, we must return the newly created object:
        return obj

    def __array_finalize__(self, obj):
        print("RealisticInfoArray.__array_finalize__")
        # see InfoArray.__array_finalize__ for comments
        if obj is None:
            return
        self.info = getattr(obj, "info", None)

In [44]:
arr = np.arange(5)
# creates ndarray from existing ndarray
obj = RealisticInfoArray(arr, info="information")
print("obj:", type(obj), obj)
print("obj info", obj.info)

# new-from-template
print("\n--- new from template ---")
v = obj[1:]
print("v:", type(v), v)
print("v info", v.info)

# view casting
print("\n--- view casting ---")
arr = np.arange(10)
cast_arr = arr.view(RealisticInfoArray)
print("cast_arr:", type(cast_arr), cast_arr)
print("cast_arr info", cast_arr.info)

RealisticInfoArray.__new__
RealisticInfoArray.__array_finalize__
obj: <class '__main__.RealisticInfoArray'> [0 1 2 3 4]
obj info information

--- new from template ---
RealisticInfoArray.__array_finalize__
v: <class '__main__.RealisticInfoArray'> [1 2 3 4]
v info information

--- view casting ---
RealisticInfoArray.__array_finalize__
cast_arr: <class '__main__.RealisticInfoArray'> [0 1 2 3 4 5 6 7 8 9]
cast_arr info None


# Typing
- numpy.ndarray[Shape, DataType]
- numpy.typing.NDArray[DataType] - any shape, specified type
    - `NDArrayInt = npt.NDArray[np.int_]`
- numpy.typing.ArrayLike - for converting lists, tuples, etc into ndarrays

``` python
def as_array(a: npt.ArrayLike) -> np.ndarray:
    return np.array(a)
```

- numpy.typing.DTypeLike - for converting dtype-ish things into dtypes

``` python
def as_dtype(d: npt.DTypeLike) -> np.dtype:
    return np.dtype(d)
```

# Iteration

In [6]:
import numpy as np

a = np.arange(20).reshape((4, 5))
print(np.nditer(a))
for v in np.nditer(a):
    print("v:", v)
    # print("type v", type(v))

v: 0
v: 1
v: 2
v: 3
v: 4
v: 5
v: 6
v: 7
v: 8
v: 9
v: 10
v: 11
v: 12
v: 13
v: 14
v: 15
v: 16
v: 17
v: 18
v: 19


In [17]:
it = np.nditer(a, flags=["multi_index"])
for v in it:
    print("it.multi_index", it.multi_index)
    x, y = it.multi_index
    print("x", a[x, y])

it.multi_index (0, 0)
x 0
it.multi_index (0, 1)
x 1
it.multi_index (0, 2)
x 2
it.multi_index (0, 3)
x 3
it.multi_index (0, 4)
x 4
it.multi_index (1, 0)
x 5
it.multi_index (1, 1)
x 6
it.multi_index (1, 2)
x 7
it.multi_index (1, 3)
x 8
it.multi_index (1, 4)
x 9
it.multi_index (2, 0)
x 10
it.multi_index (2, 1)
x 11
it.multi_index (2, 2)
x 12
it.multi_index (2, 3)
x 13
it.multi_index (2, 4)
x 14
it.multi_index (3, 0)
x 15
it.multi_index (3, 1)
x 16
it.multi_index (3, 2)
x 17
it.multi_index (3, 3)
x 18
it.multi_index (3, 4)
x 19


In [4]:
import numpy as np

a = np.arange(20).reshape((2, 10))
print(a[0, 9])
print(a)

9
[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]]
